## Dissertation Project - News Summary Dissertation [100 marks]

### Motivation 

> 1. Provide tools for anyone needing to speed up their research process
> 2. Providing ways for user to quickly determine whether a piece of research is beneficial for their specific search terms




In [1]:
# from IPython.display import HTML

# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Toggle code"></form>''')

### Imports

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import string
import random
from tqdm.notebook import tqdm
import ipywidgets as widgets
import fitz
import fasttext

import numpy as np 
import pandas as pd
import math
import time

import json
import pickle
import _pickle as pickle

import transformers
from simplet5 import SimpleT5
from transformers import AutoTokenizer,BertTokenizer,T5Tokenizer,T5Config

DATASET = "./Dataset/"

C:\Users\tomwu\AppData\Roaming\Python\Python39\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
Global seed set to 42


Detect which device (CPU/GPU) to use.

In [2]:
seed = 0
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed) 
torch.cuda.manual_seed_all(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


### Overview

Summarisation model
> 1. Dataset preprocessing
> 2. Dataloader
> 3. RNN model definition
> 4. Model training
> 5. Model prediction evaluation
> 6. Dataset Exploration
> 7. Dataset modification/Data Augmentation
> 8. Model improvement
> 9. Model finalisation and evaluation

Paper querying
> 1. Attention on query (Return usefulness percentage
> 2. Evaluate performance


# Data preprocessing

In [4]:
# # Comment this block if dataset is reorganised
# DATA_DIR = "SSN/papers.SSN.jsonl"
# dataset_path = DATASET+DATA_DIR
# with open(dataset_path) as f:
#     lines = f.read().splitlines()
# df_inter = pd.DataFrame(lines)
# df_inter.columns = ['json_element']
# df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))
# df_final.to_json("./Dataset/SSN/SSN_Dataset.json")
# df_final

In [5]:
# for i,summary in df_final.iterrows():
#     temp = summary["abstract"]
#     print(type(summary["abstract"]))
#     break

In [6]:
# #Comment this block if dataset is shortened
# DATA_DIR = "SSN/SSN_Dataset.json"
# dataset_path = DATASET+DATA_DIR
# df = pd.read_json(dataset_path)
# df

In [7]:
# df.iloc[140794]["section_names"]

Function to return index for conclusion section

In [8]:
# def trim_index(row):
#     return [row.index(x)+1 for x in row if x.startswith('conclusion') or x.startswith("summar")]

In [9]:
# def trim_text(text, index):
#     return text[0:index]

In [10]:
# # Comment this block if dataset is shortened
# # Trim text after conclusion
# indexes = []
# for i, row in df.iterrows():
#     section = row["section_names"]
#     #print(section)
#     index = trim_index(section)
#     #print(index)
#     if not index:
#         indexes.append(i)
#     # if section can be filtered
#     else:
#         index = index[0]
#         abstract = row["abstract"]
#         text = row["text"]
#         section = row["section_names"]
#         df.at[i, "section_names"] = trim_text(section, index)
#         df.at[i, "abstract"] = trim_text(abstract, index)
#         df.at[i, "text"] = trim_text(text, index)
# # dropping rows in dataframe that can't easily filter out reference section
# print(len(indexes))
# df.drop(indexes, inplace=True)
# df.to_json("./Dataset/SSN/SSN_Dataset_Short.json")

35% of paper will be removed from the dataset due to it not having conclusion(s) and summary(ies) in their section titles, making it difficult to filter out the reference and appendix text

In [11]:
# DATA_DIR = "SSN/SSN_Dataset_Short.json"
# dataset_path = DATASET+DATA_DIR
# df = pd.read_json(dataset_path)
# df.reset_index(drop=True, inplace=True)
# df

See diversity in papers, select only computer science

In [12]:
# plot_points = df['domain'].value_counts()
# plot_points

In [13]:
# df = df.loc[df['domain'].isin([['Computer science']])]
# df.to_json("./Dataset/SSN/SSN_Dataset_CompSci_Short.json")
# df

In [14]:
# DATA_DIR = "cnn_dailymail/train.csv"
# dataset_path = DATASET+DATA_DIR
# df = pd.read_csv(dataset_path)
# df = df.rename(columns={'article': 'text', 'highlights': 'abstract'})
# df

In [15]:
# DATA_DIR = "SSN/SSN_Dataset_CompSci_Short.json"
# dataset_path = DATASET+DATA_DIR
# df = pd.read_json(dataset_path)
# df.reset_index(drop=True, inplace=True)
# df

In [16]:
# summary_df = df[["abstract", "text"]]
# summary_df

Check if any columns contain empty values

In [17]:
# summary_df.isnull().any()

\'hi\' \ is used for space <br>
random space, which is used for reference [15] quote box <br>
sec ref is hyperlink to a section <br>
fig ref is hyperlink to a figure <br>
inlineform <br>
displayform are both symbols, both contains numbers in string <br>
remove all forms and remove all symbols but keep numbers <br>

In [3]:
# def contain_let(string):
#     return any(char.isalpha() for char in string)

In [4]:
# def contain_num(string):
#     return any(char.isdigit() for char in string)

In [5]:
# def contain_special(string, allowed):
#     '''
#     allowed is a list containing allowed symbols to pass detection
#     '''
#     return any(not(char.isalpha() or char.isdigit()) and (char not in allowed) for char in string)

In [6]:
# def cleanLine(line, text=True, aug=False):
#     "text parameter is to indicate whether the line is from text or abstract"
#     symbols = ["'", "’"]
#     stop_words = list(ENGLISH_STOP_WORDS)
    
#     clean_line = line.lower()

#     # fix apostrophes in line by removing apostrophe with no following alphabet character
#     clean_line = clean_line.replace("'", " ")
#     clean_line = clean_line.replace(",", " ")
# #     # remove apostrophe if last character is apostrophe
# #     if clean_line and (clean_line[-1] == "'"):
# #         clean_line = clean_line[0:len(clean_line)-1]
# #     # fix apostrophes in line by removing space before single quote
# #     clean_line = clean_line.replace(" '", "'")


#     # clean line = clean line remove forms
#     words = clean_line.split()
#     #  remove forms, words with special characters inside
#     # if contain letter and number
#     # if contain special character not in allowed symbols and removing punctuations
#     # then remove
#     words = [x.replace(x, "") if (contain_let(x) and contain_num(x))
#              or contain_special(x, symbols)
#              else x for x in words]

#     # remove empty strings
#     words = filter(None, words)

#     # stop words from sklearn, remove stop words
#     if text:
#         words = [x for x in words if not x in stop_words]
#     # remove from line randomly
#     if text and aug:
#         choices = random.choices(words, k=math.floor(len(line)*0.2))
#         words.remove(choices)
        
#      # combine the items into 1 string
#     clean_line = ' '.join(words)

    

#     return clean_line

In [22]:
# def concatParagraph(paragraph, text=True):
#     clean_paragraph = ""
#     for line in paragraph:
#         lines = cleanLine(line)
#         clean_paragraph += cleanLine(lines, text) + " "
#         #print(clean_paragraph)
        
#     return(clean_paragraph.strip())

In [23]:
# def concatPaper(paper, text=True):
#     clean_paper = ""
#     for paragraph in paper:
#         clean_paper += concatParagraph(paragraph, text) + " "
#     return(clean_paper.strip())

In [24]:
# import time
# interval = 0.0001
# aug_chance = 0.5
# for i, row in tqdm(summary_df.iterrows(), total=summary_df.shape[0]):
#     abstract = row["abstract"]
#     paper = row["text"]
    
# #     summary_df.at[i, "abstract"] = concatParagraph(abstract, text=False)
# #     summary_df.at[i, "text"] = concatPaper(paper)
#     if random.random() > aug_chance:
#         row_val = cleanLine(paper, text=True, aug=True)
#         new_row = pd.Series([abstract, row_val], index=summary_df.columns)
#         df_summary = df_summary.append(row1,ignore_index=True) 
#         df_summary
#     summary_df.at[i, "abstract"] = cleanLine(abstract, text=True)
#     summary_df.at[i, "text"] = cleanLine(paper)
#     time.sleep(interval)

In [25]:
# summary_df.to_csv("./Dataset/cnn_dailymail/cleaned_cnn_train.csv")
# summary_df

In [3]:
# DATA_DIR = "cnn_dailymail/cleaned_cnn_train_150_short3.csv"
# dataset_path = DATASET+DATA_DIR
# df = pd.read_csv(dataset_path)
# df = df[["source_text", "target_text"]]
# df

In [3]:
DATA_DIR = "SSN\SSN_Dataset_CompSci_Short_Clean_HalfStop_170_Filter5.json"
dataset_path = DATASET+DATA_DIR
df = pd.read_json(dataset_path)
#df = df[["source_text", "target_text"]]
# df = df.rename(columns={'text': 'source_text', 'abstract': 'target_text'})
#df['source_text'] = "summarize: " + df['source_text']
df

,target_text,source_text
0,tracking developments highly dynamic data tech...,summarize: ubiquity online resources massive g...
1,paper propose new method enhance mapping paral...,summarize: large scale graph based application...
2,investigate models mitogenactivated protein ki...,summarize: mathematical modelling intra cellul...
3,machine learning used number security related ...,summarize: nowadays machine learning used numb...
4,propose novel pose robust spatial aware gan ps...,summarize: work solve makeup transfer task tra...
...,...,...
7982,image restoration extensively researched topic...,summarize: image denoising problem researched ...
7983,present neural encoder decoder model convert i...,summarize: optical character recognition ocr c...
7984,ability detect pedestrians moving objects cruc...,summarize: autonomous cars currently primed ma...
7985,goal paper use multi task learning efficiently...,summarize: slot filling models useful method s...


In [12]:
# t_max = 0
# a_max = 0
# for i, row in tqdm(df.iterrows(), total=len(df)):
#     t_len = len(tokeniser(row['source_text'])['input_ids'])
#     a_len = len(tokeniser(row['target_text'])['input_ids'])
#     if t_len > t_max:
#         t_max = t_len
#     if a_len > a_max:
#         a_max = a_len
# print("t_max: ",t_max)
# print("a_max: ", a_max)

  0%|          | 0/7987 [00:00<?, ?it/s]

t_max:  2800
a_max:  200


In [6]:
# len(indexes)

2163

In [7]:
# df.drop(indexes, inplace=True)
# df.reset_index(drop=True, inplace=True)
# df.to_json(DATASET+"SSN\SSN_Dataset_CompSci_Short_Clean_HalfStop_170_Filter5.json")
# df

,target_text,source_text
0,tracking developments highly dynamic data tech...,summarize: ubiquity online resources massive g...
1,paper propose new method enhance mapping paral...,summarize: large scale graph based application...
2,investigate models mitogenactivated protein ki...,summarize: mathematical modelling intra cellul...
3,machine learning used number security related ...,summarize: nowadays machine learning used numb...
4,propose novel pose robust spatial aware gan ps...,summarize: work solve makeup transfer task tra...
...,...,...
7982,image restoration extensively researched topic...,summarize: image denoising problem researched ...
7983,present neural encoder decoder model convert i...,summarize: optical character recognition ocr c...
7984,ability detect pedestrians moving objects cruc...,summarize: autonomous cars currently primed ma...
7985,goal paper use multi task learning efficiently...,summarize: slot filling models useful method s...


In [4]:
# ABSTRACT_MAX = 170
# TEXT_MAX = 3050
ABSTRACT_MAX = 200
TEXT_MAX = 2800

In [11]:
# tokeniser = AutoTokenizer.from_pretrained('google/t5-efficient-small') 
# # takes into account of apostrophe

In [8]:
# text = df.at[0, "target_text"]
# print(len(tokeniser(text)["input_ids"]))
# for i in tokeniser(text)["input_ids"]:
#     print(i,tokeniser.decode(i))
# tokeniser.decode(tokeniser(text)["input_ids"])

In [32]:
# rows = []
# t_max= 2524
# a_max=  118
# t_min=  668
# a_min=  39
# for i, row in df.iterrows():
#     t_word = len(row["text"].split())
#     a_word = len(row["abstract"].split())
#     if ((t_word > t_max or t_word < t_min) or (a_word > a_max or a_word < a_min)):
#         rows.append(i)

# df.drop(rows, inplace=True)
# df.reset_index(drop=True, inplace=True)
# df.to_json(DATASET+"SSN/SSN_Dataset_CompSci_Short_Clean_HalfStop_180.json")
# df

Convert DF to text

In [25]:
# text_txt = df['text']
# #text_txt = text_txt.str.removeprefix("summarize: ")
# text_txt.to_csv("text_full_cnn.txt", header=False,index=False)
# # write to file. 

In [26]:
# vocab_model = fasttext.train_unsupervised('text_full_cnn.txt', minn=2, epoch=10)
# vocab_model.save_model("fastText_full_NoStop.bin")

# Dataloader

Split dataset

In [5]:
train_data, test_data = train_test_split(df, random_state=seed, train_size = 0.7)
train_data, valid_data = train_test_split(train_data, random_state=seed, train_size=0.8)

In [6]:
model = SimpleT5()
model.load_model(model_type="t5", model_dir="outputs/best", use_gpu=True)

In [ ]:
model.train(train_df=train_data,
        eval_df=valid_data,
        source_max_token_len=TEXT_MAX,
        target_max_token_len=ABSTRACT_MAX,
        batch_size=2,
        max_epochs=8,
        use_gpu=True,)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 31.2 M
-----------------------------------------------------
31.2 M    Trainable params
0         Non-trainable params
31.2 M    Total params
124.882   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

C:\Users\tomwu\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
C:\Users\tomwu\anaconda3\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

In [7]:
model_config = model.model.config.to_dict()
for k, v in model_config.items():
    print(k, v)
print("----------------")
# model_config = model.model.config.to_dict()
# model_config["num_heads"] = 8
#model_config["dropout_rate"] = 0.2
# model_config["d_model"] = 512
model_config["temperature"] = 1 #1 or >1 for more random
model_config["do_sample"] = True
config = T5Config(**model_config)
# model = AutoModelForSeq2SeqLM.from_config(config)
model.model.config = config
model.model.config

vocab_size 32128
d_model 512
d_kv 64
d_ff 2048
num_layers 6
num_decoder_layers 6
num_heads 8
relative_attention_num_buckets 32
dropout_rate 0.1
layer_norm_epsilon 1e-06
initializer_factor 1.0
feed_forward_proj relu
use_cache True
return_dict True
output_hidden_states False
output_attentions False
torchscript False
torch_dtype float32
use_bfloat16 False
pruned_heads {}
tie_word_embeddings True
is_encoder_decoder True
is_decoder False
cross_attention_hidden_size None
add_cross_attention False
tie_encoder_decoder False
max_length 20
min_length 0
do_sample False
early_stopping False
num_beams 1
num_beam_groups 1
diversity_penalty 0.0
temperature 1.0
top_k 50
top_p 1.0
repetition_penalty 1.0
length_penalty 1.0
no_repeat_ngram_size 0
encoder_no_repeat_ngram_size 0
bad_words_ids None
num_return_sequences 1
chunk_size_feed_forward 0
output_scores False
return_dict_in_generate False
forced_bos_token_id None
forced_eos_token_id None
remove_invalid_values False
architectures ['T5ForConditionalGen

In [19]:
for i,row in valid_data.iterrows():
    inputs = row["source_text"]
    output = model.predict(inputs,length_penalty=30,num_beams=25, repetition_penalty=2.5)
    print(row["target_text"])
    print("-----------------")
    print(output)
    print("")
    break
    
# inputs = valid_data.at[0,"source_text"]
# print(inputs)
# print("")
# print(valid_data.at[0,"target_text"])
# model.device = "cpu"
# model.predict(inputs,max_length=100)

people ask questions far richer informative creative current ai systems propose neural program generation framework modeling human question asking represents questions formal programs generates programs encoder decoder based deep neural network extensive experiments using information search game method ask optimal questions synthetic settings predict questions humans likely ask unconstrained settings propose novel grammar based question generation framework trained reinforcement learning able generate creative questions supervised data
-----------------
['paper introduce neural program generation framework modeling human behavior rich question asking task generating creative human like questions propose novel reinforcement learning framework generating creative human like questions information search game called battleship player uses convolutional encoder represent game state transformer decoder model generates interesting creative human like questions using context free grammar']



In [8]:
import gc
summaries = pd.DataFrame()
for i, row in tqdm(test_data.iterrows(), total=len(test_data)):
    prediction = model.predict(row["source_text"],length_penalty=30,num_beams=20, repetition_penalty=2.5)
    try:
        prediction = prediction[0]
    except:
        pass
    summaries.at[i, "actual"] = row["target_text"]
    summaries.at[i, "predict"] = prediction
    del prediction
    torch.cuda.empty_cache()
    gc.collect()
summaries

  0%|          | 0/2397 [00:00<?, ?it/s]

,actual,predict
4008,automatic fake news detection challenging prob...,paper introduces liar new dataset automatic fa...
6227,introduce novel unsupervised domain adaptation...,paper introduce novel domain adaptation paradi...
5328,introduce continuous domain framework recovery...,paper introduce continuous domain framework re...
1774,text based information retrieval ir systems in...,recent years deerwester et al recently propose...
1633,recent research shown performance search perso...,paper propose new embedding approach construct...
...,...,...
5095,recurrent sequence generators conditioned inpu...,paper propose novel end end trainable speech r...
7653,strabismus influential ophthalmologic diseases...,paper propose tele strabismus dataset establis...
7408,pre trained word vectors ubiquitous natural la...,paper propose modification cbow skip gram enri...
2148,natural language inference nli known recognizi...,paper propose discourse marker augmented netwo...


In [12]:
copycat = summaries.copy(deep=True)
copycat.reset_index(drop=True, inplace=True)
copycat
copycat.to_json(DATASET+"SSN\SSN_Dataset_CompSci_test_predictions.json")

In [14]:
summary = pd.read_json(DATASET+"SSN\SSN_Dataset_CompSci_test_predictions.json")
summary

,actual,predict
0,automatic fake news detection challenging prob...,paper introduces liar new dataset automatic fa...
1,introduce novel unsupervised domain adaptation...,paper introduce novel domain adaptation paradi...
2,introduce continuous domain framework recovery...,paper introduce continuous domain framework re...
3,text based information retrieval ir systems in...,recent years deerwester et al recently propose...
4,recent research shown performance search perso...,paper propose new embedding approach construct...
...,...,...
2392,recurrent sequence generators conditioned inpu...,paper propose novel end end trainable speech r...
2393,strabismus influential ophthalmologic diseases...,paper propose tele strabismus dataset establis...
2394,pre trained word vectors ubiquitous natural la...,paper propose modification cbow skip gram enri...
2395,natural language inference nli known recognizi...,paper propose discourse marker augmented netwo...


In [22]:
from rouge_score import rouge_scorer
scores = []
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
for i, row in tqdm(summary.iterrows(), total=len(summary)):
    target = row["actual"]
    prediction = row["predict"]
    temp_score = scorer.score(target, prediction)
    scores.append(temp_score)

  0%|          | 0/2397 [00:00<?, ?it/s]

In [45]:
scores

[{'rouge1': Score(precision=0.4166666666666667, recall=0.24390243902439024, fmeasure=0.30769230769230765),
  'rouge2': Score(precision=0.19148936170212766, recall=0.1111111111111111, fmeasure=0.140625),
  'rougeL': Score(precision=0.2708333333333333, recall=0.15853658536585366, fmeasure=0.19999999999999998)},
 {'rouge1': Score(precision=0.6666666666666666, recall=0.41237113402061853, fmeasure=0.5095541401273885),
  'rouge2': Score(precision=0.3389830508474576, recall=0.20833333333333334, fmeasure=0.25806451612903225),
  'rougeL': Score(precision=0.4166666666666667, recall=0.25773195876288657, fmeasure=0.3184713375796178)},
 {'rouge1': Score(precision=0.6153846153846154, recall=0.3516483516483517, fmeasure=0.4475524475524476),
  'rouge2': Score(precision=0.3333333333333333, recall=0.18888888888888888, fmeasure=0.24113475177304966),
  'rougeL': Score(precision=0.4423076923076923, recall=0.25274725274725274, fmeasure=0.32167832167832167)},
 {'rouge1': Score(precision=0.225, recall=0.11392

In [26]:
rouge1 = []
rouge2 = []
rougeL = []
for score in scores:
    rouge1.append(score['rouge1'])
    rouge2.append(score['rouge2'])
    rougeL.append(score['rougeL'])

In [37]:
def get_avg_rouge(rouge):
    p = []
    r = []
    f = []
    for ro in rouge:
        p.append(ro.precision)
        r.append(ro.recall)
        f.append(ro.fmeasure)
    return (np.average(p), np.average(r), np.average(f))

In [38]:
r1 = get_avg_rouge(rouge1)
r2 = get_avg_rouge(rouge2)
rl = get_avg_rouge(rougeL)

In [42]:
r1

(0.42044247379899363, 0.28032399160544325, 0.3281029090376607)

In [43]:
r2

(0.17317429450904487, 0.11449963736257739, 0.1343177695415752)

In [44]:
rl

(0.28202568524191646, 0.187763536977355, 0.2197271764850558)